In [1]:
import json
import pandas as pd

In [2]:
JSON_PATH = '../Data/Processed/custom.geo.json'
CSV_PATH = '../Data/Processed/'

In [3]:
f = open(JSON_PATH)
data = json.load(f)

In [4]:
df_2005 = pd.read_csv(CSV_PATH + "data_2005.csv")
list(df_2005['Country name'])

['Afghanistan',
 'Angola',
 'Albania',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bosnia and Herzegovina',
 'Belarus',
 'Belize',
 'Bolivia, Plurinational State of',
 'Brazil',
 'Bhutan',
 'Botswana',
 'Central African Republic',
 'Canada',
 'Switzerland',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Congo, The Democratic Republic of the',
 'Congo',
 'Colombia',
 'Comoros',
 'Costa Rica',
 'North Cyprus',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Germany',
 'Djibouti',
 'Denmark',
 'Dominican Republic',
 'Algeria',
 'Ecuador',
 'Egypt',
 'Spain',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'United Kingdom',
 'Georgia',
 'Ghana',
 'Guinea',
 'Gambia',
 'Greece',
 'Guatemala',
 'Guyana',
 'Hong Kong',
 'Honduras',
 'Croatia',
 'Haiti',
 'Hungary',
 'Indonesia',
 'India',
 'Ireland',
 'Iran, Islamic Republic of',
 'Iraq',
 'Iceland',


In [5]:
a = data['features'][3]['properties']

In [6]:
replacements = {}
replacements['Central African Republic'] = "Central African Rep."
replacements['Congo, The Democratic Republic of the'] = "Dem. Rep. Congo"
# not in our dataset : Eritrea
# not in our dataset : Guinea-Bissau
# not in our dataset : Eq. Guinea
# not in our dataset : W. Sahara
replacements['South Sudan'] = "S. Sudan"
# not in our dataset : Swaziland
replacements['Tanzania, United Republic of'] = "Tanzania"
# not in our dataset : Brunei
replacements['North Cyprus'] = "N. Cyprus"
replacements['Iran, Islamic Republic of'] = "Iran"
# not in our dataset : Somaliland
replacements['Korea, Republic of'] = "Korea"
replacements["Lao People's Democratic Republic"] = "Lao PDR"
# not in our dataset : Dem. Rep. Korea aka North korea
replacements['Syrian Arab Republic'] = "Syria"
# not in our dataset : Timor-Leste
replacements['Viet Nam'] = "Vietnam"
replacements['Taiwan, Province of China'] = "Taiwan"
replacements['Bolivia, Plurinational State of'] = "Bolivia"
# not in our dataset : Falkland Is.
replacements['Venezuela, Bolivarian Republic of'] = "Venezuela"
# not in our dataset : Bahamas
replacements['Palestine, State of'] = "Palestine"
replacements['Dominican Republic'] = "Dominican Rep."
# not in our dataset : Greenland
# not in our dataset : Puerto Rico
replacements['Bosnia and Herzegovina'] = "Bosnia and Herz."
replacements['North Macedonia'] = "Macedonia"
replacements['Czechia'] = "Czech Rep."
replacements['Moldova, Republic of'] = "Moldova"
replacements['Russian Federation'] = "Russia"
# not in our dataset : Fiji
# not in our dataset : New Caledonia
# not in our dataset : Papua New Guinea
# not in our dataset : Solomon Is.
# not in our dataset : Vanuatu
countries_to_ignore = [
     "Eritrea",
     "Guinea-Bissau",
     "Eq. Guinea",
     "W. Sahara",
     "Swaziland",
     "Brunei",
     "Somaliland",
     "Dem. Rep. Korea",
     "Timor-Leste",
     "Falkland Is.",
     "Bahamas",
     "Greenland",
     "Puerto Rico",
     "Fiji",
     "New Caledonia",
     "Papua New Guinea",
     "Solomon Is.",
     "Vanuatu"
]

In [7]:
def make_names_same_as_json(df):
    for old in replacements:
    #print(f"old: {old} new : {replacements[old]}")
        df = df.replace(old, replacements[old])
    return df

In [8]:
## create the updated csv files with the names corresponding to the json
for year in years:
    df = pd.read_csv(CSV_PATH + f"data_{year}.csv")
    df = make_names_same_as_json(df)
    df.to_csv(f'{CSV_PATH}/data_{year}_updated.csv')

NameError: name 'years' is not defined

In [9]:
## Check that only countries not in dataset are here
df_test = pd.read_csv(CSV_PATH + "data_2021_updated.csv")
for country in data['features']:
    if len(df_test[df_test['Country name'] == country['properties']['name']].index) != 1:
        print(f"name in json : {country['properties']['name']}")

name in json : Eritrea
name in json : Guinea-Bissau
name in json : Eq. Guinea
name in json : W. Sahara
name in json : Swaziland
name in json : Brunei
name in json : Somaliland
name in json : Dem. Rep. Korea
name in json : Timor-Leste
name in json : Falkland Is.
name in json : Bahamas
name in json : Greenland
name in json : Puerto Rico
name in json : Fiji
name in json : New Caledonia
name in json : Papua New Guinea
name in json : Solomon Is.
name in json : Vanuatu


In [10]:
'''
Life Ladder
Log GDP per capita
Social support
Healthy life expectancy at birth
Freedom to make life choices
Generosity
Perceptions of corruption
Alcohol consumption
Happiness/GDP cap.
'''
def clean_new_properties(d):
    target_properties = ["Life Ladder", "Log GDP per capita", 
                         "Social support", 
                         "Healthy life expectancy at birth", 
                         "Freedom to make life choices", 
                         "Generosity", 
                         "Perceptions of corruption", 
                         "Alcohol consumption", 
                         "Happiness/GDP cap."]
    new_dict = {}
    for p in target_properties:
        value = list(d[p].values())
        if len(value) > 0:
            new_dict[p] = value[0]
        else:
            print("here")
            new_dict[p] = smallest
    return new_dict

In [12]:
f = open(JSON_PATH)
years = range(2005, 2021 + 1)
for year in years:
    data = json.load(f)
    df = pd.read_csv(CSV_PATH + f"data_{year}_updated.csv")
    for idx, country in enumerate(data['features']):
        country_name = country['properties']['name']
        if country_name not in countries_to_ignore:
            properties_to_add = json.loads(df[df['Country name'] == country_name].to_json())
            new_properties = {**country['properties'], **clean_new_properties(properties_to_add)}
            data['features'][idx]['properties'] = new_properties
    
    with open(f"../Data/Processed/new_{year}.geo.json", 'w') as fp:
        json.dump(data, fp)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)